# SAT Solver Assignment
Group 69<br>
**Knowledge Representation**<br>
November 2024

****

## Preperations

*Everything we need to have before we can build the SAT Solver.*

**Imports** - Use this cell for all imports.

In [71]:
# import modules
import math
from tqdm import tqdm

**Data** - Import the needed data. The data needs to be in DIMACS format. Each clause in DIMACS CNF consists of:

- A series of positive or negative literals (numbers) that represent a Boolean variable or its negation.
- Each clause ends with 0.
- Each variable in DIMACS CNF is a positive integer, with unique integers representing each Boolean variable.

For Sudoku, we use a triplet `(r, c, n)`:
- `r` for the row (1 to 9)
- `c` for the column (1 to 9)
- `n` for the number (1 to 9)

In [72]:
# Geeft een file met 91 sudoku's terug, sudokus[0] is de eerste sudoku met len(sudokus[0]) == 81 (9x9=81 karakters)
with open('./top91.sdk.txt') as f:
    sudokus = f.read().splitlines()

*You are free to choose any programming language you fancy, but we must be able to run your SAT solver with the command SAT -Sn inputfile , for example: SAT -S2 sudoku_nr_10 , where SAT is the (compulsory) name of your program, n=1 for the basic DP and n=2 or 3 for your two other strategies, and the input file is the concatenation of all required input clauses (in your case: sudoku rules + given puzzle).*

****

## Part 1 - Building the SAT Solver

**The Rules of Sudoku**
1. Each number (1-9) must appear exactly once in each row.
2. Each number must appear exactly once in each column.
3. Each number must appear exactly once in each 3x3 subgrid.

**The Data of Sudoku in DIMACS**

In [73]:
def load_rules(filename: str) -> list[list[int]]:
    with open(filename) as f:
        raw_rules = f.read().splitlines()

    rules = [list(map(int, clause.split()))[:-1] for clause in raw_rules[1:]]
    return rules

def load_sudoku(filename: str) -> list[list[list[int | str]]]:
    with open(filename) as f:
        sudoku = f.read().splitlines()

    puzzles: list[list[list[int | str]]] = []
    
    puzzledim: int = math.sqrt(len(sudoku[0]))
    assert puzzledim.is_integer(), "The sudoku is not square"
    puzzledim = int(puzzledim)

    for puzzle in sudoku:
        newpuzzle = []
        for r in range(puzzledim):
            row = []
            for c in range(puzzledim):
                val = puzzle[r * puzzledim + c]
                val = int(val) if val != '.' else 0 # convert to int if we know the value
                row.append(val)
            newpuzzle.append(row)
        puzzles.append(newpuzzle)
    return puzzles

**Functions to Assign Obvious Values** - Fill in the single and pure literal clauses.

In [74]:
def single_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]] | None:
    """
    If a clause has only one literal, assigns it to satisfy the clause, removes satisfied clauses, 
    and removes negated literals from the remaining clauses. Repeats until no single-literal 
    clauses remain or an unsatisfiable clause (empty clause) is found.

    Args:
        clauses (list[list[int]]): List of clauses, each a list of integers (literals).
        assignment (dict[int, bool]): Current variable assignments, aka the sudoku.

    Returns:
        list[list[int]] | None: Updated list of clauses, or None if unsatisfiable.
    """
    while any(len(clause) == 1 for clause in clauses):
        for clause in clauses:
            if len(clause) == 1:
                literal = clause[0]
                assignment[abs(literal)] = literal > 0
                print(f"Assigning single literal: {literal} => {assignment[abs(literal)]}")

                new_clauses = []
                for c in clauses:
                    if literal in c:
                        continue
                    updated_clause = [l for l in c if l != -literal]
                    if not updated_clause:
                        print(f"Conflict! Clause: {c}")
                        return None
                    new_clauses.append(updated_clause)
                clauses = new_clauses
                break
    return clauses


def pure_literal(clauses: list[list[int]], assignment: dict[int, bool]) -> list[list[int]]:
    """
    Assigns values to pure literals (literals with no negation in any clause) and updates clauses.

    Args:
        clauses (List[List[int]]): List of clauses, each as a list of literals.
        assignment (Dict[int, bool]): Dictionary of current variable assignments, aka the sudoku.

    Returns:
        List[List[int]]: Updated list of clauses with pure literals removed.
    """
    # Create a set of all unique literals in the clauses
    literals = {lit for clause in clauses for lit in clause}
    
    # Identify pure literals (those without their negation in any clause)
    pure_literals = [l for l in literals if -l not in literals]

    # Assign values to pure literals and remove them from the clauses
    for literal in pure_literals:
        # Assign the value True for positive literals, False for negative ones
        assignment[abs(literal)] = literal > 0
        
        # Remove clauses containing the pure literal
        clauses = [clause for clause in clauses if literal not in clause]

    return clauses

**DPLL Algorithm** - DPLL recursive algorithm function,  without heuristics, to perform the fucntions above and recursively gives values to literals.

In [75]:
def dpll(rules: list[list[int]], unassigned: dict[int, bool], assigned: dict[int, bool], size, pbar: tqdm, history: list[int] = [0]) -> dict[int, bool] | None:
    """
    Solves a set of clauses using the DPLL algorithm.

    Args:
        rules (list[list[int]]): A list of clauses, each as a list of literals.
        unassigned (dict[int, bool]): Variables that need to be solved.
        assigned (dict[int, bool]): Fixed variables that cannot be changed.
        size (int): Size of the grid (e.g., for Sudoku).
        pbar (tqdm): Progress bar for the solver.
        history (list[int]): List of literals representing the path of the solver.

    Returns:
        dict[int, bool] | None: Satisfying assignment if one exists, otherwise None.
    """
    print("Start DPLL with history:", history)

    puzzle_dict = dict()
    puzzle_dict.update(unassigned)
    puzzle_dict.update(assigned)

    # Simplify the clauses using unit propagation and pure literal elimination
    clauses = single_literal(rules, puzzle_dict)
    if clauses is None:
        return None  # Conflict found
    if not clauses:
        return puzzle_dict  # All clauses satisfied, return combined solution

    clauses = pure_literal(clauses, puzzle_dict)
    if not clauses:
        # print(clauses)
        return puzzle_dict  # All clauses satisfied, return combined solution
    
    # print('clauses are:', clauses)
    
    # Filter out variables that are already assigned True in the unassigned list
    unassigned_vars = [var for var in unassigned if unassigned[var] is None]
    unique_vars = list(set(unassigned_vars))

    if not unique_vars:
        print("No unassigned variables left")
        return None  # No unassigned variables left

    # Select the first unassigned literal
    literal = unique_vars[0]
    print(f"Assigning {literal} = True")
    
    puzzle_dict[literal] = True
    updated_rules = [c for c in clauses if literal in c]  # Remove satisfied clauses
    # print(updated_rules) TODO: Something here does not work properly, but it could also be in single/pure literals
    result = dpll(updated_rules, unassigned.copy(), assigned, size, pbar, history + [literal])
    if result:
        return result  # If assignment works, return it

    # If assigning True doesn't work, backtrack and try False
    puzzle_dict[literal] = False
    updated_clauses = [c for c in clauses if -literal not in c]  # Remove satisfied clauses by -literal
    print(f"Assigning {literal} = False")
    result = dpll(updated_clauses, unassigned.copy(), assigned, size, pbar, history + [-literal])
    if result:
        return result  # If assignment works, return it

    # Backtrack
    print(f"No solution for {literal}, backtracking...")
    unassigned.pop(literal)  # Reset unassigned value
    return None


*give (2)+(3) as input to (1) and return the solution to the given puzzle. This output should again be a DIMACS file, but containing only the truth assignment to all variables (729 for Sudoku, different for other SAT problems). If your input file is called 'filename', then make sure your outputfile is called 'filename.out'. If there is no solution (inconsistent problem), the output can be an empty file. If there are multiple solutions (eg. non-proper Sudoku) you only need to return a single solution.*

**Heuristics** - Two different heuristics of our choice

In [76]:
# heuristic one

In [77]:
# heuristic two

**SAT Solver** - Solves the Sudoku

Extra information about the code:
Each cell in a Sudoku puzzle can be represented by a unique variable based on its row, column, and possible number. The encoding formula for each variable is:

`variable=(i+1)×100+(j+1)×10+value`<br>

where:<br>

`i is the row index (0 to 8)`<br>
`j is the column index (0 to 8)`<br>
`value is the number in the cell (1 to 9)`<br>

This formula converts each cell and its possible values into unique numbers by combining the row, column, and value in a single integer. For example, if a cell at row 0, column 0 can be a 5, the encoded variable would be:<br>

`variable=(0+1)×100+(0+1)×10+5=115`<br>

In [78]:
def solve_sudoku(rules: list[list[int]], puzzle: list[list[int]], size) -> list[list[int]] | None:
    """
    Solves a Sudoku puzzle using given rules and an initial puzzle setup.

    Args:
        rules (list[list[int]]): A list of rules in CNF form for Sudoku constraints.
        puzzle (list[list[int]]): A grid of given size representing the puzzle, with `0` for empty cells.

    Returns:
        list[list[int]] | None: A solved Sudoku grid if a solution exists, otherwise None.
    """
    assigned = {}  # Pre-filled cells
    unassigned = {}  # To solve

    for i, row in enumerate(puzzle):
        for j, value in enumerate(row):
            if value != 0:  # Pre-filled
                literal = (i + 1) * 100 + (j + 1) * 10 + value
                assigned[literal] = True
            else:
                for k in range(1, size + 1):
                    literal = (i + 1) * 100 + (j + 1) * 10 + k
                    unassigned[literal] = None  # Unsolved initially

    # print(f"Initial Assigned: {assigned}")
    # print(f"Initial Unassigned: {unassigned}")

    with tqdm(total=2**len(unassigned), desc="Solving Sudoku") as pbar:
        solution = dpll(rules, unassigned, assigned, size, pbar)
    
    # print("Initial Clauses:", rules)
    # print("Initial Assigned Vars:", assigned)
    # print("Initial Unassigned Vars:", unassigned)
    return solution

**Test Sudoku** - Try and see if it works

In [79]:
load_sudoku('./4x4_easy.txt')[1]

[[1, 2, 3, 4], [3, 4, 1, 0], [2, 1, 4, 3], [4, 3, 0, 1]]

In [80]:
# Load the Sudoku rules and the puzzle
filename = './sudoku-rules-4x4.txt'
rules = load_rules(filename)

# Load the puzzles from the file and select the second puzzle (index 1)
puzzle = load_sudoku('./4x4_easy.txt')[1]
print(puzzle)

size = 4  # Size of the grid (4x4 Sudoku)

# Solve the Sudoku puzzle
solution = solve_sudoku(rules, puzzle, size)

print('solution:', solution)

# If a solution is found, print the solved Sudoku grid
if solution:
    result = [[0] * size for _ in range(size)]
    
    for var, value in solution.items():
        if value:
            i = (var // 100) - 1
            j = ((var // 10) % 10) - 1
            num = var % 10

            if 0 <= i < size and 0 <= j < size:
                result[i][j] = num
            else:
                print(f"Index out of range: i={i}, j={j}")
    
    print("Solved Sudoku:", result)
else:
    print('No solution')

[[1, 2, 3, 4], [3, 4, 1, 0], [2, 1, 4, 3], [4, 3, 0, 1]]


Solving Sudoku:   0%|          | 0/256 [00:00<?, ?it/s]

Start DPLL with history: [0]
Assigning 431 = True
Start DPLL with history: [0, 431]
solution: {241: None, 242: None, 243: None, 244: None, 431: True, 432: True, 433: True, 434: True, 111: True, 122: True, 133: True, 144: True, 213: True, 224: True, 231: True, 312: True, 321: True, 334: True, 343: True, 414: True, 423: True, 441: True, 131: True, 421: True, 331: True, 341: True, 411: True}
Solved Sudoku: [[1, 2, 1, 4], [3, 4, 1, 0], [2, 1, 1, 1], [1, 1, 4, 1]]
